<div style="color:pink">
hier möchte ich einmal versuchen, die Pixel, die von Bots gesetzt wurden, nachzuzeichnen
</div>

In [1]:
#alle Schritte, die nötig waren, um die Bots einzustufen
from src.data.dataset_functions import get_dataframe

dataFrame = get_dataframe(True)

import pyspark.sql.functions as F
timeDiffFrame = dataFrame.alias('df1').join(dataFrame.alias('df2'),F.col('df1.user_id') == F.col('df2.user_id'),'inner')
timeDiffFrame = timeDiffFrame.where('df2.t > df1.t')
timeDiffFrame = timeDiffFrame.orderBy(['df1.user_id', 'df1.t'])
timeDiffFrame = timeDiffFrame.select(F.col('df1.user_id'),F.col('df1.t').alias('t1'),F.col('df2.t').alias('t2'),(F.col('df2.t') - F.col('df1.t')).alias('diff'))
timeDiffReduced = timeDiffFrame.dropDuplicates(['user_id', 't1'])
timeDiffReduced = timeDiffReduced.orderBy(['user_id', 't1'])

botDiff = timeDiffReduced.alias('tdr1').join(timeDiffReduced.alias('tdr2'),F.col('tdr1.user_id') == F.col('tdr2.user_id'),'inner')
botDiff = botDiff.where('tdr1.diff == tdr2.diff').where('tdr1.t2 == tdr2.t1')
botDiff = botDiff.select('tdr1.user_id','tdr1.diff')
botDiff = botDiff.dropDuplicates(['user_id'])
botDiff = botDiff.orderBy(['user_id'])


providing ../data/raw/2022_place_canvas_history-000000000050.csv ...
../data/raw/2022_place_canvas_history-000000000050.csv is already in data/raw
:: loading settings :: url = jar:file:/usr/local/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/vscode/.ivy2/cache
The jars for the packages stored in: /home/vscode/.ivy2/jars
graphframes#graphframes added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-a3f23bf0-ccec-4f95-a4e8-f596ccac4caf;1.0
	confs: [default]
	found graphframes#graphframes;0.8.2-spark3.2-s_2.12 in spark-packages
	found org.slf4j#slf4j-api;1.7.16 in central
:: resolution report :: resolve 68ms :: artifacts dl 2ms
	:: modules in use:
	graphframes#graphframes;0.8.2-spark3.2-s_2.12 from spark-packages in [default]
	org.slf4j#slf4j-api;1.7.16 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   2   |   0   |   0   |   0   ||   2   |   0   |
	----------------------------------

22/06/21 17:56:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/06/21 17:56:15 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [70]:
botsTotal = dataFrame.alias('df').join(botDiff.alias('bd'),F.col('df.user_id') == F.col('bd.user_id'), 'inner')
botsTotal = botsTotal.select(F.col('df.user_id'),F.col('df.x'),F.col('df.y'),F.col('df.t'),F.col('pixel_color'))
botsTotal = botsTotal.orderBy(['user_id', 't'])
botsTotal = botsTotal.dropDuplicates(['user_id', 't']) #jeder Nutzer kann zu jeder Zeit nur einen Pixel gesetzt habe
botsTotal2 = botsTotal.select(F.col('x'),F.col('y'),F.col('pixel_color'))


In [75]:
from PIL import Image, ImageColor

#def newImg():
 #      img = Image.new("RGB", (2000,2000), "WHITE")
  #     img.save('../reports/figures/bots-canvas.jpg')
   #    for row in botsTotal2:
    #     color = ImageColor.getrgb("orange")
     #    img.putpixel(('x','y'), color)
      # return img

img = Image.new("RGB", (2000,2000), "WHITE")
img.save('../reports/figures/bots-canvas.jpg')      
for row in botsTotal2:
  color = ImageColor.getrgb('pixel_color')
  img.putpixel(('x','y'), color)
img.show()


#for row in dataCollect:
 #   x,y,count = list(row)
  #  pixelcoords = '('+str(x)+','+str(y)+')'
   # plt.bar(pixelcoords,count,color='#5050cc', width=0.5)
                           

ValueError: unknown color specifier: 'pixel_color'